# 라이브러리, 데이터셋
- - -

In [11]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 기본 사용법
- - -

### 구성 단계(construction phase)

In [2]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.], [2.]])

product = tf.matmul(matrix1, matrix2)

### 실행 단계(execution phase)

In [3]:
sess = tf.Session()

result = sess.run(product)
print(result)

sess.close()

[[12.]]


In [4]:
with tf.Session() as sess:
    print(sess.run(matrix1))
    print(sess.run(matrix2))

[[3. 3.]]
[[2.]
 [2.]]


### 연산 도구 설정

In [5]:
%%time
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        print(sess.run(product))

[[12.]]
CPU times: user 6 ms, sys: 2.77 ms, total: 8.77 ms
Wall time: 6.27 ms


In [6]:
%%time
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        print(sess.run(product))

[[12.]]
CPU times: user 5.8 ms, sys: 3.01 ms, total: 8.81 ms
Wall time: 6.43 ms


### Variables

In [7]:
# 값이 0인 스칼라로 초기화된 변수를 만든다
state = tf.Variable(0, name="counter")

# 'state'에 1을 더하는 작업을 만든다
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# 그래프 작동 후에는 init 작업을 실행해서 변수 초기화한다
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(state))
    
    for _ in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


### Fetches

In [9]:
%%time
input1 = tf.constant([3.0])
input2 = tf.constant([2.0])
input3 = tf.constant([5.0])

intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)

[array([21.], dtype=float32), array([7.], dtype=float32)]
CPU times: user 13.2 ms, sys: 3.51 ms, total: 16.7 ms
Wall time: 16.3 ms


### Feeds

- 'feed 메커니즘'은 텐서플로우에서 graph의 연산에서 직접 tensor 값을 줄 수 있다. 

In [10]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = input1 * input2

sess = tf.Session()

print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))

sess.close()

[array([14.], dtype=float32)]


# MNIST 초급
- - -

In [23]:
# 변수 정의

x = tf.placeholder(tf.float32, [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])

In [24]:
# 손실 함수, 최적화 알고리즘 정의

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [30]:
# 학습
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [36]:
# 모델 평가하기

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9196


# MNIST 고급
- - -

### 단일 신경망

In [37]:
sess = tf.InteractiveSession()

In [39]:
# placeholder, 변수 정의, 변수 초기화

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

In [40]:
# 손실 함수, 모델 훈련

y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [45]:
# 모델 평가

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9165


### 다중 신경망

In [ ]:
# 함수, 